# Caso 1: Open-Meteo
Neste caso práctico imos obter datos meteorolóxicos desde unha API pública (Open-Meteo). O proxecto terá os seguintes compoñentes:
- Produtor Kafka. Programa en Python que accederá á API a través do módulo *requests* e producirá os resultados nun *topic* ao que accederá despois o noso programa en Spark.
- Programa en Spark. Programa que recompila os datos de Kafka e realiza agregacións por xanelas de tempo. En función de onde se envíen os resultados teremos dúas variantes:
  - Á consola.
  - A ficheiro Parquet.
- Por último, teremos un programa *echo* en *Streamlit* que mostrará de forma visual os datos recompilados.

## productor_open_meteo.py
Necesitamos as seguintes importacións:
- **KafkaProducer** do módulo **kafka**.
- **dumps** do paquete **json**.
- **requests**: para realizar peticións á API.
- **time**.
- **datetime**.

Os pasos a seguir serán os seguintes:
1. Crear un produtor Kafka.
2. Facer unha chamada á API de Open-Meteo.
3. Transformar os datos a JSON e formatealos.
4. Usar o método `send()` da clase *KafkaProducer* para escribir os datos nun *topic*.

O primeiro será crear o produtor Kafka indicando o **bootstrap-server** e a codificación.


In [ ]:
from kafka import KafkaProducer
producer = KafkaProducer(
    bootstrap_servers=['kafka:9092'],
    value_serializer=lambda x: dumps(x).encode('utf-8')
)

Imos empregar a API de Open-Meteo. O primeiro paso sería consultar a [documentación oficial](https://open-meteo.com/en/docs). Na sección **API Documentation** vemos un listado dos parámetros dispoñibles, entre eles *latitude* e *longitude*, que son obrigatorios.

Vemos que as variables están distribuídas en varios grupos: *daily weather*, *hourly weather*, *current weather*... Interésanos este último.

Para comunicarnos coa API imos empregar o módulo **requests**. Especificamos a URL da API e os parámetros (pares clave-valor) e chamamos ao método **get()**. Convertimos a resposta a *json* para explorar a súa estrutura:


In [1]:
import requests
import json

url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 42.33669,        # Coordenadas de Ourense
    "longitude": -7.86407,
    "current_weather": "true"
}

response = requests.get(url, params=params)

if response.ok:
    data = response.json()
    print(json.dumps(data, indent=2))
else:
    print("Erro:", response.status_code)

{
  "latitude": 42.3125,
  "longitude": -7.875,
  "generationtime_ms": 0.06401538848876953,
  "utc_offset_seconds": 0,
  "timezone": "GMT",
  "timezone_abbreviation": "GMT",
  "elevation": 147.0,
  "current_weather_units": {
    "time": "iso8601",
    "interval": "seconds",
    "temperature": "\u00b0C",
    "windspeed": "km/h",
    "winddirection": "\u00b0",
    "is_day": "",
    "weathercode": "wmo code"
  },
  "current_weather": {
    "time": "2026-02-10T11:30",
    "interval": 900,
    "temperature": 16.2,
    "windspeed": 13.6,
    "winddirection": 230,
    "is_day": 1,
    "weathercode": 61
  }
}


Unha vez que temos a resposta, o seguinte paso é obter os datos que nos interesan, no noso caso *current_weather*. Engadímoslle o nome da cidade e un *timestamp* normalizado.


In [2]:


from datetime import datetime
data = response.json()["current_weather"]

data["city"]="Ourense"
data["timestamp"] = datetime.utcnow().isoformat()
print(data)

{'time': '2026-02-10T11:30', 'interval': 900, 'temperature': 16.2, 'windspeed': 13.6, 'winddirection': 230, 'is_day': 1, 'weathercode': 61, 'city': 'Ourense', 'timestamp': '2026-02-10T11:43:56.073797'}


Unha vez que teñamos os datos no formato que queremos, almacenados nunha variable (no exemplo *data*), escribímolos no *topic* indicado:


In [ ]:
from kafka import KafkaProducer
#...

producer.send("topic",data)

Pódese facer o mesmo con varias localizacións empregando un bucle.

## Programa Spark
A estrutura do programa en *Spark* é moi similar á dos notebooks vistos previamente:
1. Inicializamos a sesión de Spark: temos que incluír os paquetes necesarios para usar *Kafka*.
2. Definimos o esquema do *DataFrame* inicial.
3. Definimos o fluxo de entrada. Neste caso temos que indicar, entre outros, o **bootstrap-server** e o nome do **topic** do que se lerán os datos.
4. Creamos un *DataFrame* novo transformando os datos iniciais. Hai que ter en conta as limitacións para distintos tipos de agregacións (especificadas no documento 00 e na [documentación oficial](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)).
5. Iniciamos o procesamento, como sempre, indicando o destino e o modo.

### Exemplo de inicialización da sesión de Spark:

```python
spark = SparkSession.builder \
    .appName("OpenMeteoStreamingClean") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "2") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .getOrCreate()
```
### Exemplo de definición de esquema:
```python
spark = SparkSession.builder \
    .appName("OpenMeteoStreamingClean") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "2") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .getOrCreate()
```

### Exemplo de definición de fluxo de entrada:

```python
# Ejemplo Open-Meteo
df_kafka = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka-1:9092") \
    .option("subscribe", "open-meteo-weather") \
    .option("startingOffsets", "latest") \
    .load()

# Ejemplo general
df_kafka = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "servidor-kafka") \
    .option("subscribe", "nombre-topic") \
    .option("startingOffsets", "latest") \
    .load()

```

### Exemplo de transformación de datos:
```python
# Transformación simple: filtrado e limpieza.
df_parsed = df_kafka.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data")) \
    .select(
        col("data.city"),
        col("data.temperature"),
        col("data.windspeed"),
        col("data.winddirection"),
        to_timestamp(col("data.local_timestamp")).alias("event_time")
    )

#Transformación adicional empregando windowing e watermarking:
# Agrupación por xanela e cidade
df_grouped = df_parsed \
    .withWatermark("event_time", "2 minutes") \
    .groupBy(
        window(col("event_time"), "1 minute"),
        col("city")
    ).agg(
        avg("temperature").alias("avg_temp"),
        avg("windspeed").alias("avg_wind")
    )

```
### Exemplo de inicio do procesamento en streaming:
#### Saída a consola
```python
# Mostrar en consola
query = df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()
```

#### Saída a un arquivo parquet
```python
# Almacenar en parquet:
query=df.writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", "hdfs://spark-master:9000/user/jovyan/weather_aggregated/") \
    .option("checkpointLocation", "hdfs://spark-master:9000/user/jovyan/checkpoint_weather_aggregated/") \
    .start()
```

Neste cartafol hai exemplos de produtor e de programa en Spark. Lembrade que, para que funcione ao executalos con *spark-submit*, hai que engadir a seguinte opción:  
`--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0`
